In [2]:
# !pip3 install transformers msgspec

In [3]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from tqdm import tqdm
import msgspec
import json
import re

http_errors = [
        "400 Bad Request", "401 Unauthorized", "402 Payment Required", "403 Forbidden", "404 Not Found",
        "405 Method Not Allowed", "406 Not Acceptable", "407 Proxy Authentication Required", "408 Request Timeout",
        "409 Conflict", "410 Gone", "411 Length Required", "412 Precondition Failed", "413 Payload Too Large",
        "414 URI Too Long", "415 Unsupported Media Type", "416 Range Not Satisfiable", "417 Expectation Failed",
        "418 I'm a teapot", "421 Misdirected Request", "422 Unprocessable Entity", "423 Locked", "424 Failed Dependency",
        "425 Too Early", "426 Upgrade Required", "428 Precondition Required", "429 Too Many Requests",
        "431 Request Header Fields Too Large", "451 Unavailable For Legal Reasons", "500 Internal Server Error",
        "501 Not Implemented", "502 Bad Gateway", "503 Service Unavailable", "504 Gateway Timeout",
        "505 HTTP Version Not Supported", "506 Variant Also Negotiates", "507 Insufficient Storage",
        "508 Loop Detected", "510 Not Extended", "511 Network Authentication Required"
    ]

In [4]:
rejected = [
    'Internal Server Error',
    '__NOEDITSECTION__',
    'enter your username and password',
    'forgotten your password',
    'cookies enabled',
    'enable JavaScript in your browser.',
    'The page cannot be displayed',
    'site or edit the error_page',
    'Request unsuccessful',
]

rejected.extend(http_errors)

def replace_multiple(input_string, pattern =r"\s{6,}", replace = '   '):
    return re.sub(pattern, replace, input_string)

def replace(string):
    string = replace_multiple(string.replace('…', '.'))
    string = replace_multiple(string, pattern = r"\.{6,}", replace = '...')
    return string

def reject(string):
    if any([r in string for r in rejected]):
        return True
    return False

In [5]:
def partition(text, size = 500):
    splitted = text.split()
    return [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size)]

In [6]:
files = glob('/home/ubuntu/dedup-text-dataset/*.jsonl')
len(files)

274

In [7]:
wiki = [
    'wikipedia-2023-10-01.jsonl',
    'wikipedia-20230901.en.filtered.jsonl',
]
language_related = [
    'dictionary.jsonl',
    'dewanbahasa-jdbp.jsonl',
    'dialect.jsonl',
    'kamusbm.jsonl',
    'wiktionary-bahasa.jsonl',
]
gov_related = [
    'hansard.jsonl',
    'lom.agc.gov.my.jsonl',
    'parlimen-gov.jsonl',
    'gov.my.jsonl',
    'edu.my.jsonl',
]

In [8]:
files = wiki + language_related + gov_related
files = [os.path.join('/home/ubuntu/dedup-text-dataset', f) for f in files]
files

['/home/ubuntu/dedup-text-dataset/wikipedia-2023-10-01.jsonl',
 '/home/ubuntu/dedup-text-dataset/wikipedia-20230901.en.filtered.jsonl',
 '/home/ubuntu/dedup-text-dataset/dictionary.jsonl',
 '/home/ubuntu/dedup-text-dataset/dewanbahasa-jdbp.jsonl',
 '/home/ubuntu/dedup-text-dataset/dialect.jsonl',
 '/home/ubuntu/dedup-text-dataset/kamusbm.jsonl',
 '/home/ubuntu/dedup-text-dataset/wiktionary-bahasa.jsonl',
 '/home/ubuntu/dedup-text-dataset/hansard.jsonl',
 '/home/ubuntu/dedup-text-dataset/lom.agc.gov.my.jsonl',
 '/home/ubuntu/dedup-text-dataset/parlimen-gov.jsonl',
 '/home/ubuntu/dedup-text-dataset/gov.my.jsonl',
 '/home/ubuntu/dedup-text-dataset/edu.my.jsonl']

In [17]:
block_size = 2048

def partition(text, size = 500):
    splitted = text.split()
    return [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size)]

def read_dataset(tokenizer, train_file, block_size = block_size):
    temp = []
    
    with open(f'{train_file}.tokenized', 'w') as fopen_l:
        with open(train_file) as fopen:
            for l in tqdm(fopen):
                try:
                    l = msgspec.json.decode(l)
                except:
                    pass
                partitioned = partition(l)
                for p in partitioned:
                    tokenized = tokenizer(p)['input_ids']
                    temp.extend(tokenized)
                    while len(temp) >= block_size:
                        block = temp[:block_size]
                        temp = temp[block_size:]
                        if len(block) == block_size:
                            s = tokenizer.decode(block)
                            fopen_l.write(f'{json.dumps(s)}\n')
                            fopen_l.flush()
                        
def loop(files):
    files, _ = files
    tokenizer = AutoTokenizer.from_pretrained(
        'mistralai/Mixtral-8x7B-Instruct-v0.1',
    )
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False
    for f in files:
        read_dataset(tokenizer, f)

In [18]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [19]:
len(files)

12

In [20]:
import mp
mp.multiprocessing(files, loop, cores = len(files), returned = False)

66it [00:00, 1682.46it/s]
34192it [00:18, 1845.32it/s]
4577it [00:35, 127.73it/s]s]
1359it [00:49, 27.57it/s]s] 
54712it [01:03, 855.29it/s] 
24096it [01:11, 335.85it/s]
438316it [11:05, 658.75it/s] 
56459it [13:04, 71.97it/s] 
21590it [15:36, 23.06it/s]s]
1687it [16:07,  1.74it/s]/s]
140932it [18:58, 123.79it/s]
30055it [21:50, 22.93it/s]


In [21]:
# !git clone https://huggingface.co/datasets/malaysia-ai/online-articles-partition
!cp dedup-text-dataset/*.tokenized online-articles-partition

In [23]:
from huggingface_hub import HfApi
api = HfApi()

In [27]:
for f in files:
    f = f'{f}.tokenized'
    if not os.path.exists(f):
        continue
    
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=os.path.split(f)[1],
        repo_id='malaysia-ai/online-articles-partition',
        repo_type='dataset',
    )

wikipedia-2023-10-01.jsonl.tokenized:   0%|          | 0.00/289M [00:00<?, ?B/s]

wikipedia-20230901.en.filtered.jsonl.tokenized:   0%|          | 0.00/483M [00:00<?, ?B/s]

dictionary.jsonl.tokenized:   0%|          | 0.00/26.7M [00:00<?, ?B/s]

dewanbahasa-jdbp.jsonl.tokenized:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

dialect.jsonl.tokenized:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

...

hansard.jsonl.tokenized:   0%|          | 0.00/506M [00:00<?, ?B/s]

lom.agc.gov.my.jsonl.tokenized:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

parlimen-gov.jsonl.tokenized:   0%|          | 0.00/411M [00:00<?, ?B/s]

gov.my.jsonl.tokenized:   0%|          | 0.00/627M [00:00<?, ?B/s]

edu.my.jsonl.tokenized:   0%|          | 0.00/497M [00:00<?, ?B/s]